<a href="https://colab.research.google.com/github/SoyJimena/PIA-PROGRAMACION/blob/main/PIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import sqlite3
import datetime
import time
from sqlite3 import Error




#CREACION DE LA BASE DE DATOS

try:
    with sqlite3.connect("Ev3Modificada.db") as conn:   #INTEGER PRIMARY KEY
        mi_cursor = conn.cursor()
        mi_cursor.execute("CREATE TABLE IF NOT EXISTS clientes (clave INTEGER PRIMARY KEY, nombre TEXT NOT NULL);")
        mi_cursor.execute("CREATE TABLE IF NOT EXISTS turnos (noTurno NUMBER NOT NULL, numero, salas, turno TEXT NOT NULL, FOREIGN KEY(salas) REFERENCES salas(numero), FOREIGN KEY(salas) REFERENCES salas(nombre));")
        mi_cursor.execute("CREATE TABLE IF NOT EXISTS salas (numero INTEGER PRIMARY KEY, nombre TEXT NOT NULL, cupo NUMBER NOT NULL);")
        mi_cursor.execute("CREATE TABLE IF NOT EXISTS reservaciones (folio INTEGER PRIMARY KEY, descripcion TEXT NOT NULL, clientes INTEGER NOT NULL, turnos INTEGER NOT NULL, fecha timestamp, salas NUMBER NOT NULL, FOREIGN KEY(clientes) REFERENCES clientes(clave), FOREIGN KEY(salas) REFERENCES salas(numero), FOREIGN KEY(turnos) REFERENCES turnos(noTurno));")

        print("Las tablas han sido creada exitosamente")
        
except Error as e:
    print (e)
except:
    print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
finally:
    conn.close()
    



#INGRESAR DATOS EN LA TABLA RESERVACIONES
def rigistrar_reservacion():
    while True:
        clave_cliente= int(input("Ingrese su clave de cliente: "))
        try:
            with sqlite3.connect("Ev3Modificada.db") as conn:
                mi_cursor = conn.cursor()
                valores = {"clave":clave_cliente}
                mi_cursor.execute("SELECT * FROM clientes WHERE clave = :clave", valores)
                registro = mi_cursor.fetchall()

                if registro:
                    descripcion = input("Ingrese el nombre del evento: ")
                    numero_sala = int(input("Ingrese el numero de la sala: "))
                    try:
                        with sqlite3.connect("Ev3Modificada.db") as conn:
                            mi_cursor = conn.cursor()
                            valores = {"numero":numero_sala}
                            mi_cursor.execute("SELECT * FROM salas WHERE numero = :numero", valores)
                            nsala = mi_cursor.fetchall()
                            if nsala:
                                turnos = (input("Indique el turno que desea entre [\n 1.matutino. \n 2.vespertino. \n 3.nocturno].lower() : "))
                                try:
                                    with sqlite3.connect("Ev3Modificada.db") as conn:
                                        mi_cursor = conn.cursor()
                                        valores_turno = {"turno":turnos}
                                        mi_cursor.execute("SELECT * FROM turnos WHERE turno = :turno", valores_turno)
                                        nturno = mi_cursor.fetchall()
                                        if nturno:
                                            fecha_suma = datetime.timedelta(days=2)
                                            fecha_actual = datetime.date.today()
                                            fecha_evento_str = input ("Indique la fecha del evento en el formato dd/mm/aaaa: ")
                                            fecha = datetime.datetime.strptime(fecha_evento_str, "%d/%m/%Y").date()
                                            try:
                                                with sqlite3.connect("Ev3Modificada.db") as conn:
                                                    mi_cursor = conn.cursor()
                                                    valores = {"fecha":fecha, "salas": numero_sala, "turnos": turnos}
                                                    mi_cursor.execute("SELECT * FROM reservaciones WHERE fecha = :fecha AND salas = :salas AND turnos = :turnos", valores)
                                                    validacion = mi_cursor.fetchall()
                                                    if validacion:
                                                        print ("El registro no se puede realizar. La sala ya fue ocupada en esa fecha y turno, Lo sentimos.")
                                                    else:
                                                        if fecha <= fecha_actual + fecha_suma:
                                                            print ("No se pudo realizar la reservacion ya que debio hacerlo con dos dias de anticipacion como minimo, lo lamentamos.")
                                                        else:
                                                            try:
                                                                with sqlite3.connect("Ev3Modificada.db") as conn:
                                                                    mi_cursor = conn.cursor()
                                                                    mi_cursor.execute("SELECT folio FROM reservaciones")
                                                                    nfolio = mi_cursor.fetchall()
                                                                    if nfolio == []:
                                                                        numero_inicial = 999
                                                                        folio = numero_inicial + 1
                                                                        try:
                                                                            with sqlite3.connect("Ev3Modificada.db") as conn:
                                                                                mi_cursor = conn.cursor()
                                                                                valores_reservaciones=(folio, descripcion, clave_cliente, turnos, fecha, numero_sala)
                                                                                mi_cursor.execute("INSERT INTO reservaciones VALUES(?,?,?,?,?,?)", valores_reservaciones)
                                                                                print("Registro agregado exitosamente. Su folio de reservacion es el: ", folio)
                                                                        
                                                                        except Error as e:
                                                                            print (e)
                                                                        except:
                                                                            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                                                                        finally:
                                                                            conn.close()
                                                                    else:
                                                                        contador= (max(nfolio))
                                                                        numero_mayor = contador[0]
                                                                        folio = numero_mayor + 1
                                                                        try:
                                                                            with sqlite3.connect("Ev3Modificada.db") as conn:
                                                                                mi_cursor = conn.cursor()
                                                                                valores_reservaciones=(folio, descripcion, clave_cliente, turnos, fecha, numero_sala)
                                                                                mi_cursor.execute("INSERT INTO reservaciones VALUES(?,?,?,?,?,?)", valores_reservaciones)
                                                                                print("Registro agregado exitosamente. Su folio de reservacion es el: ", folio)
                                                                        
                                                                        except Error as e:
                                                                            print (e)
                                                                        except:
                                                                            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                                                                        finally:
                                                                            conn.close()
                                                            
                                                            except Error as e:
                                                                print (e)
                                                            except:
                                                                print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                                                            finally:
                                                                conn.close()
                                                            

                                                                    
                                            except Error as e:
                                                print (e)
                                            except:
                                                print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                                            finally:
                                                conn.close()
                                                
                                        else:
                                            print ("Debe de elegir una opcion de las anteriores.")
                                            
                                except Error as e:
                                    print (e)
                                except:
                                    print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                                finally:
                                    conn.close()
                                    
                            else:
                                print ("Esa sala no esta en existencia. Favor de registrar sala")
                                break
                    
                    except Error as e:
                        print (e)
                    except:
                        print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                    finally:
                        conn.close()
                    
                    continuar=input("Desea agregar otro registro: (si/no) ")
                    if continuar =="si":
                        continue
                    else:
                        if continuar == "no":
                            break    
                 
                else:
                    print ("Es necesario ser cliente para hacer la reservacion.")
                    break

        except Error as e:
            print (e)
        except:
            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")


#MODIFICAR RESERVACION
def modificar():
    while True:
        folio = int(input("Ingrese el numero de folio de la sala a la que desea modificar la descripcion: "))
        try:
            with sqlite3.connect("Ev3Modificada.db") as conn:
                mi_cursor = conn.cursor()
                valores_reservacion = {"folio":folio}
                mi_cursor.execute("SELECT * FROM reservaciones WHERE folio = :folio", valores_reservacion)
                noreservaciones = mi_cursor.fetchall()
                if noreservaciones:
                    try:
                        descripcion = input("Ingrese la descripcion nueva: ")
                        with sqlite3.connect("Ev3Modificada.db") as conn:
                            mi_cursor = conn.cursor()
                            valores_modificar=(descripcion, folio)
                            mi_cursor.execute("UPDATE reservaciones SET descripcion=? WHERE folio= ?", valores_modificar)
                            print("Registro modificado.")
                                
                    except Error as e:
                        print (e)
                    except:
                        print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                    finally:
                        conn.close()
                        
                else:
                    print ("El numero de folio no esta registrado como evento. Favor de revisarlo.")
                    
        except Error as e:
            print (e)
        except:
            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
        finally:
            conn.close()
                    
                    
        continuar=input("Desea modificar otro registro: (si/no) ")
        if continuar =="si":
            continue
        else:
            if continuar == "no":
                break
            
            
#DISPONIBILIDAD DE LAS SALAS
def disponibilidad ():
    while True:
        fecha_consultar = input("Ingrese la fecha que quiera consultar para ver la disponibilidad  dd/mm/aaaa/: ")
        fecha = datetime.datetime.strptime(fecha_consultar, "%d/%m/%Y").date()
        try:
            with sqlite3.connect("Ev3Modificada.db") as conn:
                mi_cursor = conn.cursor()
                valores = {"fecha":fecha}
                #mi_cursor.execute("SELECT * FROM reservaciones WHERE fecha = :fecha", valores)
                mi_cursor.execute("SELECT salas, turnos FROM reservaciones WHERE fecha = :fecha", valores)
                validacion = mi_cursor.fetchall()
                conjunto_validacion = set(validacion)
                print (conjunto_validacion)

                if validacion:
                    try:
                        with sqlite3.connect("Ev3Modificada.db") as conn:
                            mi_cursor = conn.cursor()
                            mi_cursor.execute("SELECT numero, turno FROM turnos")
                            salas = mi_cursor.fetchall()
                            if salas:
                                conjunto_salas = set(salas)
                                conjunto_disponibilidad = conjunto_salas-conjunto_validacion 
                                #print (conjunto_disponibilidad)
                                lista_disponibilidad = list(conjunto_disponibilidad)
                                ordenada = sorted(lista_disponibilidad)
                                print (ordenada)

                                break
                            
                    except Error as e:
                      print (e)
                    except:
                      print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                    finally:
                      conn.close()
        except Error as e:
          print (e)
        except:
            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
        finally:
            conn.close()
        
        
        
      
#ELIMINAR RESERVACION
def eliminar():
    while True:
        folio= int(input("Ingrese el folio de la reservacion que quiera eliminar: "))
        try:
            with sqlite3.connect("Ev3Modificada.db") as conn:
                mi_cursor = conn.cursor()
                valores = {"folio":folio}
                mi_cursor.execute("SELECT * FROM reservaciones WHERE folio = :folio", valores)
                folios = mi_cursor.fetchall()
                if folios:
                    lista_folios = folios[0]
                    fecha_s = lista_folios[4]
                    try:
                        with sqlite3.connect("Ev3Modificada.db") as conn:
                            mi_cursor = conn.cursor()
                            valores2 = {"folio":folio,"fecha": fecha_s} #"folio":folio,
                            mi_cursor.execute("SELECT * FROM reservaciones WHERE folio = :folio AND DATE(fecha) = :fecha", valores2)
                            validacion = mi_cursor.fetchall()
                            if validacion:
                                fecha_resta = datetime.timedelta(days=3)
                                fecha_actual = datetime.date.today()
                                fecha_calculo = fecha_actual + fecha_resta
                                fecha_evento = datetime.datetime.strptime(fecha_s, '%Y-%m-%d').date() 
                                print ("Tipo fecha:", type(fecha_evento))
                                if fecha_evento <= fecha_calculo:
                                    print ("No se pudo realizar la cancelacion ya que debio hacerlo con tres dias de anticipacion como minimo, lo lamentamos.")
                                else:
                                    print("Los datos encontrados con este numero de folio son: ")
                                    print(folios)
                                    respuesta = input("Una vez eliminada la reservacion no sera posible recuperarlo. ¿Esta seguro de continuar? si/no ")
                                    if respuesta == "si":
                                        try:
                                            with sqlite3.connect("Ev3Modificada.db") as conn:
                                                mi_cursor = conn.cursor()
                                                valores3=(folio,)
                                                mi_cursor.execute("DELETE FROM reservaciones WHERE folio=?", valores3)
                                                print("Reservacion eliminada.")               
                                        except Error as e:
                                            print (e)
                                        except:
                                            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                                        finally:
                                            conn.close()
                                            
                                    elif respuesta == "no":
                                        print("Volviendo al submenu...")
                                        break
                                    
                                    else:
                                        print ("No se encuentra su respuesta como opcion.")
                                        continue
                    except Error as e:
                        print (e)
                    except:
                        print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                    finally:
                        conn.close()   
                            
                else:
                    print("No se encuentra aquel folio en el registro") 
     
        except Error as e:
            print (e)
        except:
            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
        finally:
            conn.close()
        
        
        continuar=input("Desea eliminar otra reservacion: (si/no) ")
        if continuar =="si":
            continue
        else:
            if continuar == "no":
                break
       
           




    
#INGRESAR DATOS A LA TABLA SALAS
def registrar_salas():
    nombre = input("Mencione el nombre de la sala a agregar: ")
    if nombre.isspace() or len(nombre) <= 0:
      print ("Es obligatorio ingresar el nombre.")
    else:
        cupo = int(input("Ingrese el cupo de la sala: "))
        if cupo <= 0:
            print ("Es obligatorio mencionar el cupo.")
        else:
            turno1 = "matutino"
            turno2 = "vespertino"
            turno3= "nocturno"
            
            try: 
                with sqlite3.connect("Ev3Modificada.db") as conn:
                    mi_cursor = conn.cursor()
                    mi_cursor.execute("SELECT numero FROM salas")
                    nsala = mi_cursor.fetchall()
                    if nsala == []:
                        numero_inicial = 0
                        numero = numero_inicial + 1
                        try:
                            with sqlite3.connect("Ev3Modificada.db") as conn:
                                mi_cursor = conn.cursor()
                                valores = (numero, nombre, cupo,)
                                valores = (numero, nombre, cupo)
                                mi_cursor.execute("INSERT INTO salas VALUES(?,?,?)", valores)
                                print("Registro agregado exitosamente")
                                print ("El numero de la sala asignada es el: ", numero)
                        except Error as e:
                            print (e)
                        except:
                            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                        finally:
                            conn.close()
                    else:
                        contador= (max(nsala))
                        numero_mayor = contador[0]
                        numero = numero_mayor + 1
                        try:
                            with sqlite3.connect("Ev3Modificada.db") as conn:
                                mi_cursor = conn.cursor()
                                valores2 = (numero, nombre, cupo,)
                                mi_cursor.execute("INSERT INTO salas VALUES(?,?,?)", valores2)
                                print("Registro agregado exitosamente")
                                print ("El numero de la sala asignada es el: ", numero)
                        except Error as e:
                            print (e)
                        except:
                            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                        finally:
                            conn.close()
      
      
                    try:
                        with sqlite3.connect("Ev3Modificada.db") as conn:
                            mi_cursor = conn.cursor()
                            valores1 = (numero, nombre, turno1)
                            valores2 = (numero, nombre, turno2)
                            valores3 = (numero, nombre, turno3)
                            mi_cursor.execute("SELECT noTurno FROM turnos")
                            numturno = mi_cursor.fetchall()
                            if numturno == []:
                                cero = 0
                                noTurno = cero + 1
                                valores1 = (noTurno, numero, nombre, turno1)
                                valores2 = (noTurno, numero, nombre, turno2)
                                valores3 = (noTurno, numero, nombre, turno3)
                                mi_cursor.execute("INSERT INTO turnos VALUES(?,?,?,?)", valores1)
                                mi_cursor.execute("INSERT INTO turnos VALUES(?,?,?,?)", valores2)
                                mi_cursor.execute("INSERT INTO turnos VALUES(?,?,?,?)", valores3)
                            else:
                                contador= (max(numturno))
                                numero_mayor = contador[0]
                                noTurno = numero_mayor + 1
                                valores1 = (noTurno, numero, nombre, turno1)
                                valores2 = (noTurno, numero, nombre, turno2)
                                valores3 = (noTurno, numero, nombre, turno3)
                                mi_cursor.execute("INSERT INTO turnos VALUES(?,?,?,?)", valores1)
                                mi_cursor.execute("INSERT INTO turnos VALUES(?,?,?,?)", valores2)
                                mi_cursor.execute("INSERT INTO turnos VALUES(?,?,?,?)", valores3)
  
                    except Error as e:
                        print (e)
                    except:
                        print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                    finally:
                        conn.close()
                        
     

         
            except Error as e:
                print (e)
            except:
                print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
            finally:
                conn.close()

            
    
        

#INGRESAR DATOS A LA TABLA CLIENTES
def registrar_clientes ():
    while True:
        nombre = input("Mencione el nombre del cliente a agregar: ")
        if nombre.isspace() or len(nombre) <= 0:
          print ("Es obligatorio ingresar el nombre.")
          break
        else:
            try:
                with sqlite3.connect("Ev3Modificada.db") as conn:
                    mi_cursor = conn.cursor()
                    mi_cursor.execute("SELECT clave FROM clientes")
                    nclaves = mi_cursor.fetchall()
                    if nclaves == []:
                        try:
                            numero_inicial = 99
                            clave = numero_inicial + 1
                            with sqlite3.connect("Ev3Modificada.db") as conn:
                                mi_cursor = conn.cursor()
                                valores = (clave, nombre)
                                mi_cursor.execute("INSERT INTO clientes VALUES(?,?)", valores)
                                print("Registro agregado exitosamente")
                                print ("La clave del cliente asignado es el: ", clave)
                        except Error as e:
                            print (e)
                        except:
                            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                        finally:
                            conn.close()
                    else:
                        contador= (max(nclaves))
                        numero_mayor = contador[0]
                        clave = numero_mayor + 1
                        try:
                            with sqlite3.connect("Ev3Modificada.db") as conn:
                                mi_cursor = conn.cursor()
                                valores = (clave, nombre)
                                mi_cursor.execute("INSERT INTO clientes VALUES(?,?)", valores)
                                print("Registro agregado exitosamente")
                                print ("La clave del cliente asignado es el: ", clave)
                        except Error as e:
                            print (e)
                        except:
                            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                        finally:
                            conn.close()
                            
                        
            except Error as e:
                print (e)
            except:
                print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
            finally:
                conn.close()
                
            respuesta = input("¿Desea agregar otro cliente? si/no: ")
            if respuesta == "si":
                continue
            elif respuesta == "no":
                break
            else:
                print ("Su respuesta no es valida. Regresando al menu principal...")
                break
    

    

#MENU
while True:
  print("\n ** MENU PRINCIPAL **")
  print ("**********************")
  print ("1- Reservaciones")
  print ("2- Reportes")
  print ("3- Registrar una sala")
  print ("4- Registrar un cliente")
  print ("5- Salir")

  respuesta = input("Indique la opcion deseada: ")

  try:
    respuesta_int = int(respuesta)
  except ValueError:
    print ("* La respuesta no es valida*")
  except Exception:
    print("Se a presentado una excepcion: ")
    print(Exception)


   # RESERVACIONES
  if respuesta_int == 1:
      while True:
          print("\n ** SUBMENU RESERVACIONES **")
          print ("******************************")
          print ("1- Registrar nueva reservacion")
          print ("2- Modificar descripcion de una reservacion")
          print ("3- Consulta disponibilidad de salas para una fecha")
          print ("4- Eliminar una reservacion")
          print ("5- Volver al menu principal")
          
          respuesta_submenu = int(input("Indique la opcion deseada: "))
          
          try:
              respuesta_submenu_int = int(respuesta_submenu)
          except ValueError:
              print ("* La respuesta no es valida*")
          except Exception:
              print("Se a presentado una excepcion: ")
              print(Exception)
            
          if respuesta_submenu == 1:
              rigistrar_reservacion()
            
          elif respuesta_submenu == 2:
              modificar()
            
          elif respuesta_submenu == 3:
              disponibilidad ()
              
          elif respuesta_submenu == 4:
              eliminar()
            
          elif respuesta_submenu == 5:
              
              break
            
          else:
              print ("Ingrese una de las respuestas existentes en el menu.")


   #REPORTES
  elif respuesta_int == 2:
      while True:
          print ("\n ** SUBMENU REPORTES **")
          print ("************************")
          print ("1- Reporte en pantalla de reservaciones para una fecha")
          print ("2- Exportar reporte tabular en Excel")
          print ("3- Regresar al menu principal")
          
          respuesta_reportes = int(input("Indique la opcion deseada: "))
          
          try:
              respuesta_reportes_int = int(respuesta_reportes)
          except ValueError:
              print ("* La respuesta no es valida*")
          except Exception:
              print("Se a presentado una excepcion: ")
              print(Exception)
            
          if respuesta_reportes == 1:
              print("Reporte en pantalla")
            
          elif respuesta_reportes == 2:
              print ("Exportar en excel")
            
          elif respuesta_reportes == 3:
              
              break
            
          else:
              print ("Ingrese una de las respuestas existentes en el menu.")


   #REGISTRAR LA SALA
  elif respuesta_int == 3:
      registrar_salas()
    
    
    # REGISTRO CLIENTES
  elif respuesta_int == 4:
      registrar_clientes ()
            
            

  elif respuesta_int == 5:
    break

  else:
    print ("Su respuesta no corresponde a ninguna de las opciones")
    
